In [1]:
!pip install google-search-results

In [2]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
try:
    from serpapi import GoogleSearch
    print("✅ All libraries imported successfully!")
    print("✅ SerpAPI library found!")
except ImportError:
    print("❌ SerpAPI not installed!")
    print("\n📦 Run this command:")
    print("   pip install google-search-results")
    print("\nThen restart the kernel and run again.")

✅ All libraries imported successfully!
✅ SerpAPI library found!


In [9]:
SERPAPI_KEY = "5317f506f5ae86fd7689c0db2cec4cce590a26e849c125b932114cb1292d3944"

if SERPAPI_KEY == "5317f506f5ae86fd7689c0db2cec4cce590a26e849c125b932114cb1292d3944":
    print("⚠️  WARNING: You need to add your SerpAPI key!")
    print("\n📋 Quick Setup:")
    print("   1. Go to: https://serpapi.com/")
    print("   2. Sign up (free)")
    print("   3. Copy your API key from dashboard")
    print("   4. Paste it above (replace YOUR_API_KEY_HERE)")
    print("\n✅ Free tier gives you 100 searches/month")
else:
    print(f"✅ API Key configured: {SERPAPI_KEY[:10]}...{SERPAPI_KEY[-4:]}")


⚠️  WARNING: You need to add your SerpAPI key!

📋 Quick Setup:
   1. Go to: https://serpapi.com/
   2. Sign up (free)
   3. Copy your API key from dashboard
   4. Paste it above (replace YOUR_API_KEY_HERE)

✅ Free tier gives you 100 searches/month


In [10]:
class PhraseExtractor:
    """
    Extract key phrases from text for searching
    """
    
    def extract_sentences(self, text, min_words=8, max_words=15):
        """
        Extract meaningful sentences for searching
        """
        sentences = re.split(r'[.!?]+', text)
        
        key_phrases = []
        for sentence in sentences:
            words = sentence.strip().split()
            if min_words <= len(words) <= max_words:
                clean = ' '.join(words)
                key_phrases.append(clean)
        
        return key_phrases[:5]
    
    def extract_ngrams(self, text, n=4):
        """
        Extract n-grams (sequences of n words)
        """
        words = text.split()
        ngrams = []
        
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i:i+n])
            if len(ngram) > 20:
                ngrams.append(ngram)
        
        return ngrams[:10]
    
    def get_search_queries(self, text):
        """
        Generate search queries from text
        """
        queries = []
        
        sentences = self.extract_sentences(text)
        queries.extend(sentences[:3])
        
        ngrams = self.extract_ngrams(text)
        queries.extend(ngrams[:2])
        
        return queries

extractor = PhraseExtractor()
print("✅ PhraseExtractor initialized")


✅ PhraseExtractor initialized


In [11]:
class SerpAPISearcher:
    """
    Web scraper using SerpAPI - 100% reliable!
    """
    
    def __init__(self, api_key):
        self.api_key = api_key
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        print(f"✅ SerpAPI Searcher initialized")
        
        
        if api_key and api_key != "YOUR_API_KEY_HERE":
            self.test_api_key()
    
    def test_api_key(self):
        """
        Test if API key is valid
        """
        try:
            params = {
                "q": "test",
                "api_key": self.api_key,
                "num": 1
            }
            search = GoogleSearch(params)
            results = search.get_dict()
            
            if "error" in results:
                print(f"❌ API Key Error: {results['error']}")
                return False
            else:
                print("✅ API Key is valid!")
                return True
                
        except Exception as e:
            print(f"❌ API Test Failed: {str(e)}")
            return False
    
    def search_google(self, query, num_results=5):
        """
        Search Google using SerpAPI - NO BLOCKING!
        """
        try:
            print(f"🔍 Searching: '{query[:50]}...'")
            
            params = {
                "q": query,
                "api_key": self.api_key,
                "num": num_results,
                "engine": "google"
            }
            
            search = GoogleSearch(params)
            results = search.get_dict()
            
            
            if "error" in results:
                print(f"   ❌ Error: {results['error']}")
                return []
            
            
            urls = []
            for result in results.get("organic_results", []):
                url = result.get("link")
                if url and url.startswith('http'):
                    urls.append(url)
            
            print(f"   ✅ Found {len(urls)} URLs")
            return urls[:num_results]
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)[:80]}")
            return []
    
    def extract_text_from_url(self, url, timeout=10):
        """
        Extract text content from webpage
        """
        try:
            response = self.session.get(url, headers=self.headers, timeout=timeout)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            
            for element in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
                element.decompose()
            
            
            main_content = soup.find('main') or soup.find('article') or soup.find('body')
            
            if main_content:
                text = main_content.get_text(separator=' ', strip=True)
            else:
                text = soup.get_text(separator=' ', strip=True)
            
            
            text = ' '.join(text.split())
            return text[:5000]
            
        except Exception as e:
            return ""

if SERPAPI_KEY != "YOUR_API_KEY_HERE":
    scraper = SerpAPISearcher(SERPAPI_KEY)
else:
    print("⚠️  Scraper not initialized - add API key in CELL 2")

✅ SerpAPI Searcher initialized
✅ API Key is valid!


In [13]:
class WebPlagiarismDetector:
    """
    Complete web-based plagiarism detection system
    """
    
    def __init__(self, api_key):
        self.scraper = SerpAPISearcher(api_key)
        self.extractor = PhraseExtractor()
        print("✅ WebPlagiarismDetector initialized")
    
    def check_plagiarism(self, user_text, num_searches=2, results_per_search=3):
        """
        Check text against web sources using SerpAPI
        """
        print("="*70)
        print("🌐 WEB-BASED PLAGIARISM DETECTION (SerpAPI)")
        print("="*70)
        print("\n⏱️  Estimated time: 10-20 seconds")
        
        print("\n🔍 Step 1: Extracting key phrases...")
        queries = self.extractor.get_search_queries(user_text)[:num_searches]
        
        if not queries:
            print("❌ Could not extract phrases")
            return []
        
        print(f"✅ Generated {len(queries)} search queries:")
        for i, q in enumerate(queries, 1):
            preview = q[:60] + "..." if len(q) > 60 else q
            print(f"   {i}. {preview}")
        
        print(f"\n🔍 Step 2: Searching the web via SerpAPI...")
        all_urls = []
        
        for i, query in enumerate(queries, 1):
            print(f"\n   Search {i}/{len(queries)}:")
            urls = self.scraper.search_google(query, num_results=results_per_search)
            all_urls.extend(urls)
            
            if i < len(queries):
                print(f"   ⏳ Waiting 2 seconds...")
                time.sleep(2)
        
        all_urls = list(set(all_urls))[:10]
        print(f"\n✅ Total unique sources: {len(all_urls)}")
        
        if len(all_urls) == 0:
            print("\n⚠️  No URLs found")
            print("   Check your API key and internet connection")
            return []
        
        print(f"\n📄 Step 3: Fetching content from sources...")
        sources = []
        
        for i, url in enumerate(all_urls, 1):
            domain = urlparse(url).netloc
            print(f"   {i}/{len(all_urls)} {domain[:45]:<45}", end=" ")
            
            content = self.scraper.extract_text_from_url(url)
            
            if content and len(content) > 100:
                sources.append({
                    'url': url,
                    'content': content,
                    'domain': domain
                })
                print("✅")
            else:
                print("❌")
            
            time.sleep(1)
        
        print(f"\n✅ Successfully fetched {len(sources)} sources")
        
        if len(sources) == 0:
            print("\n⚠️  Could not fetch any content")
            return []
        
        print(f"\n📊 Step 4: Calculating similarity...")
        
        results = []
        user_words = set(user_text.lower().split())
        
        for source in sources:
            source_words = set(source['content'].lower().split())
            common = user_words.intersection(source_words)
            
            if len(user_words) > 0:
                similarity = (len(common) / len(user_words)) * 100
                
                results.append({
                    'url': source['url'],
                    'domain': source['domain'],
                    'similarity': similarity,
                    'common_words': len(common)
                })
        
        results = sorted(results, key=lambda x: x['similarity'], reverse=True)
        print(f"✅ Calculation complete!")
        
        return results
    
    def generate_report(self, results):
        """
        Generate plagiarism report with visualization
        """
        print("\n" + "="*70)
        print("📋 PLAGIARISM DETECTION REPORT")
        print("="*70)
        
        if not results:
            print("\n✅ No similar content found!")
            print("   Your text appears to be original.")
            return None
        
        max_similarity = max(r['similarity'] for r in results)
        
        if max_similarity >= 60:
            verdict = "🔴 HIGH RISK"
        elif max_similarity >= 40:
            verdict = "🟠 MODERATE RISK"
        elif max_similarity >= 20:
            verdict = "🟡 LOW RISK"
        else:
            verdict = "🟢 MINIMAL RISK"
        
        print(f"\n{verdict}")
        print(f"📊 Highest Similarity: {max_similarity:.1f}%")
        print(f"🔍 Sources Checked: {len(results)}")
        
        print(f"\n🔗 Top {min(5, len(results))} Matching Sources:")
        print("-"*70)
        
        for i, result in enumerate(results[:5], 1):
            print(f"\n{i}. {result['domain']}")
            print(f"   Similarity: {result['similarity']:.1f}%")
            print(f"   Common words: {result['common_words']}")
            print(f"   URL: {result['url'][:65]}...")
        
        # Create visualization
        if len(results) > 0:
            df = pd.DataFrame(results[:10])
            
            plt.figure(figsize=(12, 7))
            
            top_results = results[:min(8, len(results))]
            domains = [r['domain'][:35] for r in top_results]
            similarities = [r['similarity'] for r in top_results]
            
            colors = ['red' if s >= 60 else 'orange' if s >= 40 else 'yellow' if s >= 20 else 'green' 
                      for s in similarities]
            
            plt.barh(domains, similarities, color=colors, edgecolor='black', linewidth=1.2)
            plt.xlabel('Similarity Score (%)', fontsize=12, fontweight='bold')
            plt.title('Web-Based Plagiarism Detection Results (SerpAPI)', fontsize=14, fontweight='bold')
            plt.xlim(0, 100)
            
            plt.axvline(x=60, color='red', linestyle='--', alpha=0.4, linewidth=2, label='High (60%)')
            plt.axvline(x=40, color='orange', linestyle='--', alpha=0.4, linewidth=2, label='Moderate (40%)')
            plt.axvline(x=20, color='yellow', linestyle='--', alpha=0.4, linewidth=2, label='Low (20%)')
            
            plt.legend(loc='lower right', fontsize=10)
            plt.grid(axis='x', alpha=0.3, linestyle=':')
            plt.tight_layout()
            plt.show()
            
            return df
        
        return None

if SERPAPI_KEY != "YOUR_API_KEY_HERE":
    detector = WebPlagiarismDetector(SERPAPI_KEY)
else:
    print("⚠️  Detector not initialized - add API key in CELL 2")

✅ SerpAPI Searcher initialized
✅ API Key is valid!
✅ WebPlagiarismDetector initialized


In [16]:
sample_text = """
Your Text
"""

if SERPAPI_KEY != "YOUR_API_KEY_HERE":
    print("🧪 TESTING WEB PLAGIARISM CHECKER")
    print("="*70)
    print(f"\nTest Text ({len(sample_text.split())} words):")
    print(sample_text)
    print("\n" + "="*70)
    
    print("\n⏳ Starting check...\n")
    
    results = detector.check_plagiarism(
        sample_text,
        num_searches=2,
        results_per_search=3
    )
    
    if results and len(results) > 0:
        print("\n" + "="*70)
        print("🎉 CHECK SUCCESSFUL!")
        print("="*70)
        report_df = detector.generate_report(results)
        
        print("\n📊 Summary Statistics:")
        print(f"   • Total sources: {len(results)}")
        print(f"   • Highest similarity: {max(r['similarity'] for r in results):.1f}%")
        print(f"   • Average similarity: {sum(r['similarity'] for r in results)/len(results):.1f}%")
        print(f"   • Sources above 40%: {sum(1 for r in results if r['similarity'] >= 40)}")
        
        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if report_df is not None and not report_df.empty:
            output_file = f"plagiarism_results_{timestamp}.csv"
            report_df.to_csv(output_file, index=False)
            print(f"\n💾 Results saved to: {output_file}")
    else:
        print("\n⚠️  No results obtained")
else:
    print("⚠️  Please add your SerpAPI key in CELL 2 first!")

🧪 TESTING WEB PLAGIARISM CHECKER

Test Text (2 words):

Your Text



⏳ Starting check...

🌐 WEB-BASED PLAGIARISM DETECTION (SerpAPI)

⏱️  Estimated time: 10-20 seconds

🔍 Step 1: Extracting key phrases...
❌ Could not extract phrases

⚠️  No results obtained
